In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import re
import pickle
import time

# 깃허브에서 참고한 코드

In [ ]:
class Naver:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('headless')
        self.driver = ""
        self.url_list =[]
        self.stop_crawling = True
    def Open_Chrome(self, URL) :  
        self.driver = webdriver.Chrome("/Users/sky/Downloads/chromedriver",options=self.options)
        self.driver.get(URL)
        self.driver.implicitly_wait(3)
    
    def Close_Chrome(self):
        self.driver.close()
        
    def get_url(self,xpath):
        get_xpath= self.driver.find_element_by_xpath(xpath)
        url =  get_xpath.get_attribute('href')
        return url
    
    def loop_get_url(self, limit_num, save_file_name):
        i = 1
        j = 1
        num = len(self.url_list)

        while (j<=5) :
            try :
                xpath= '//*[@id="section_body"]/ul[' + str(j) + ']/li[' + str(i) + ']/dl/dt[2]/a'
                self.url_list.append(self.get_url(xpath))
                i= i+1
                num = num +1
                if(num==limit_num):
                    self.stop_crawling = False
                    break
            except:
                i = 1
                j = j+1
        with open(save_file_name, 'wb') as f:
            pickle.dump(self.url_list, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
naver = Naver()
i = 1 
while naver.stop_crawling:
    print("current page : ",i)
    url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105#&date=%2000:00:00&page='+str(i)
    naver.Open_Chrome(url)
    naver.loop_get_url(1000,'it.pickle')
    naver.Close_Chrome()
    i = i+1
    time.sleep(.5)
    print("current list length: ",len(naver.url_list))
print(len(naver.url_list))

In [ ]:
with open(r'/Users/sky/class_python/practice/과학.pickle', 'rb') as f:

    data = pickle.load(f, encoding='utf-8')

In [ ]:
data

# URL(참고코드)로 데이터 크롤링

In [ ]:
import requests
import time
from selenium import webdriver

wd = webdriver.Chrome('/Users/sky/Downloads/chromedriver')

naver = []

for url in data:
    wd.get(url)
    
    soup = BeautifulSoup(wd.page_source)
    
    news = {}
    news['title']= soup.select('#articleTitle')[0].text
    news['date'] = soup.select('#main_content > div.article_header > div.article_info > div > span')[0].text
    news['link'] = soup.select('#main_content > div.article_header > div.article_info > div > a')[0].attrs['href']
    news['content']= soup.select('#articleBodyContents')[0].text
    
    time.sleep(.5)
    #break
    naver.append(news)
    

In [ ]:
import pandas as pd
it_news = pd.DataFrame(naver)
it_news.to_csv('it_news_1126.csv', encoding='utf8')